In [ ]:
#Importar bibliotecas
import numpy as np
from pathlib import Path
import ccp
import xlwings as xw
from scipy.optimize import differential_evolution

Q_ = ccp.Q_
xw.pro = True

data_dir = Path.cwd()

ccp.__version__full

'ccp: 0.3.4 | CP : 6.8.0 | REFPROP : 10.0.0.99'

## Definindo a pressão a montante e jusante do compressor

In [2]:
Ps_1sec_target = Q_(1.54,"kgf/cm²").to("Pa")
Pd_2sec_target = Q_(22.29,"kgf/cm²").to("Pa")

# Composição do Fluido de Processo

In [ ]:
fluid={
        "HYDROGEN": 4.85,
        "NITROGEN": 0.97,
        "CARBON MONOXIDE": 0.08,
        "OXYGEN":0.05 ,
        "METHANE": 18.08,
        "ETHYLENE": 4.41,
        "ETHANE": 7.05,
        "CARBON DIOXIDE": 0.31,
        "HYDROGEN SULFIDE": 0.970,
        "PROPENE": 15.07,
        "PROPANE": 7.11,
        "I-BUTANE": 4.81,
        "I-BUTENE": 3.68,
        "1-BUTENE": 2.3+0.29,
        "cis-2-Butene": 1.87+2.6,
        "N-BUTANE": 2.3,
        "I-PENTANE": 2.48+1.63+1.85,
        "N-HEXANE": 9.9,
        "N-PENTANE": 1.27,
        "WATER": 4.6,
    }

#### Retirando a água da composição do fluido para calcular a vazão seca (Dry Flow) e assim realizar o cálculo da fração de condensado (que ocorre no trocador de calor entre 1ª e 2ª seção)

In [ ]:
fluid_dry={
        "HYDROGEN": 4.85,
        "NITROGEN": 0.97,
        "CARBON MONOXIDE": 0.08,
        "OXYGEN":0.05 ,
        "METHANE": 18.08,
        "ETHYLENE": 4.41,
        "ETHANE": 7.05,
        "CARBON DIOXIDE": 0.31,
        "HYDROGEN SULFIDE": 0.970,
        "PROPENE": 15.07,
        "PROPANE": 7.11,
        "I-BUTANE": 4.81,
        "I-BUTENE": 3.68,
        "1-BUTENE": 2.3+0.29,
        "cis-2-Butene": 1.87+2.6,
        "N-BUTANE": 2.3,
        "I-PENTANE": 2.48+1.63+1.85,
        "N-HEXANE": 9.9,
        "N-PENTANE": 1.27,
        #"WATER": 4.6,
    }

In [5]:
#Normalizando as frações do fluido retirando a Água!

# Soma total original
total = sum(fluid_dry.values())

# Normalização para 100%
fluid_dry = {k: round((v / total) * 100, 2) for k, v in fluid_dry.items()}

# Impressão do resultado
print("Composição normalizada (soma = 100%):")
for comp, val in fluid_dry.items():
    print(f"{comp}: {val}%")


Composição normalizada (soma = 100%):
HYDROGEN: 5.16%
NITROGEN: 1.03%
CARBON MONOXIDE: 0.09%
OXYGEN: 0.05%
METHANE: 19.25%
ETHYLENE: 4.69%
ETHANE: 7.51%
CARBON DIOXIDE: 0.33%
HYDROGEN SULFIDE: 1.03%
PROPENE: 16.04%
PROPANE: 7.57%
I-BUTANE: 5.12%
I-BUTENE: 3.92%
1-BUTENE: 2.76%
cis-2-Butene: 4.76%
N-BUTANE: 2.45%
I-PENTANE: 6.35%
N-HEXANE: 10.54%
N-PENTANE: 1.35%


### Construindo o estado do gás na entrada do compressor - 1ªSeção

In [6]:
# 1ª SEÇÃO
suc_1sec = ccp.State(
    p=Ps_1sec_target, # original set 1.59 kgf/cm²
    T=Q_(39, "degC"),
    fluid=fluid_dry     
)

##### Construindo a classe Impeller que calcula as condições de descarga de cada seção baseado nas curvas de performance do fabricante do compressor (curva de Head e Eficiência)

In [7]:
imp_1sec = ccp.Impeller.load_from_engauge_csv(
    suc=suc_1sec,
    curve_name="curve_1sec_custom",
    curve_path=data_dir,
    b=Q_(5, "mm"),
    D=Q_(294, "mm"),
    head_units="kJ/kg",
    flow_units="m³/h",
    power_losses_units="kW",
    number_of_points=5,
)

## Restrições: Pressão de descarga na 2ª Seção, perda de carga no trocador de calor e Rotação do compressor

In [8]:
Perda_de_carga = Q_(0.51,"kgf/cm²").to("Pa")

speed = Q_(5354,"rpm").to("rad/s")

args = [Pd_2sec_target.m, Perda_de_carga.m, speed.m]

## Função objetivo

In [9]:
def update_flow(x, *args):
    
    flow_m_1 = x[0]
    speed = args[2]
    point_1sec = imp_1sec.point(flow_m= flow_m_1, speed=speed)
    Pd_1sec = point_1sec.disch.p().m
    Pd_2sec = args[0]
    press_drop = args[1]               ## perda de carga no trocador de calor
    Ps_2sec = Pd_1sec - press_drop   

    disch = point_1sec.disch
    disch.update(p=Ps_2sec, T=Q_(40, "°C"))
    gas_rate = disch.Q()                                                                         #Fração mássica de gás após condensação
    base_fluid_2sec = {k: v for k, v in zip(disch.fluid_names(), disch.mole_fractions_vapor())}         #composição do gás após condensação

    new_suc_2sec = ccp.State(
        p=Ps_2sec, # original set 5.48 kgf/cm²
        T=Q_(40, "degC"),
        fluid=base_fluid_2sec
    )

    new_imp_2sec = ccp.Impeller.load_from_engauge_csv(
        suc=new_suc_2sec,
        curve_name="curve_2sec_custom",
        curve_path=data_dir,
        b=Q_(5, "mm"),
        D=Q_(294, "mm"),
        head_units="kJ/kg",
        flow_units="m³/h",
        power_losses_units = "kW",
        number_of_points=5,
    )
    
    flow_m_2 = gas_rate*x[0]
    new_point_2sec= new_imp_2sec.point(flow_m= flow_m_2, speed=speed)
    Pd_2_calc = new_point_2sec.disch.p().to("Pa").m
    Pd_2_calc = round(Pd_2_calc/100,0)                                          #mudando para hPa e arredondando para a convergencia não ficar caótica
    Pd_2sec = round(Pd_2sec/100,0)

    mod = abs(Pd_2_calc- Pd_2sec)

    return mod

# Lista para registrar histórico
history = []

# Callback que salva o histórico e para se f(x) <= 0
def save_history_and_stop(xk, convergence):
    fx = update_flow(xk,*args)
    history.append((xk.copy(), fx))
    
    # Interrompe a otimização quando f(x) <= 0
    return fx <= 1


# LIMITES 
bounds = [(imp_1sec.curves[1].points[0].flow_m.m, imp_1sec.curves[1].points[-1].flow_m.m)]


# Executar otimização
res = differential_evolution(
    update_flow,
    bounds=bounds,
    args=args,
    strategy='best1bin',
    maxiter=1000,
    tol = 0.1,
    polish=True,
    disp=True,
    callback=save_history_and_stop,
)

# Resultado
print("\nResultado final com differential_evolution:")
print("x*: ", res.x)
print("f(x*): ", res.fun)

# Histórico
print("\nHistórico de iterações:")
for i, (x_val, f_val) in enumerate(history):
    print(f"Iter {i+1}: x = {x_val}, f(x) = {f_val}")

    

c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 3.140 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 2.841 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 3.248 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 3.946 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-p

differential_evolution step 1: f(x)= 309.0


c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 5.591 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 5.601 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 6.259 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 5.790 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-p

differential_evolution step 2: f(x)= 309.0


c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 6.627 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 6.228 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 5.581 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 5.799 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-p

differential_evolution step 3: f(x)= 309.0


c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 5.773 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 6.286 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 6.217 m³/s
  warnings.warn(
c:\Users\UT6E\AppData\Local\miniforge3\envs\py312\Lib\site-packages\ccp\impeller.py:361: UserWarning: Expected point is being extrapolated.
Interpolation limits: 3.973 m³/s ~ 5.577 m³/s
Expected point flow: 5.960 m³/s
  warnings.warn(


differential_evolution step 4: f(x)= 274.0
differential_evolution step 5: f(x)= 58.0
differential_evolution step 6: f(x)= 0.0
Polishing solution with 'L-BFGS-B'

Resultado final com differential_evolution:
x*:  [51.93546067]
f(x*):  0.0

Histórico de iterações:
Iter 1: x = [52.01576394], f(x) = 309.0
Iter 2: x = [52.01576394], f(x) = 309.0
Iter 3: x = [52.01576394], f(x) = 309.0
Iter 4: x = [51.86395516], f(x) = 274.0
Iter 5: x = [51.9202314], f(x) = 58.0
Iter 6: x = [51.93546067], f(x) = 0.0


## Imprimindo Resultados

In [10]:
flow_m_1 = res.x[0]
speed = imp_1sec.curves[1].speed
point_1sec = imp_1sec.point(flow_m= flow_m_1, speed=speed)
Pd_1sec = point_1sec.disch.p().m
Pd_2sec = args[0]
Ps_2sec = Q_((Pd_1sec - 50013.915), "Pa")   ### 0.51 kgf/cm² = 61781.895 pascal

disch = point_1sec.disch
disch.update(p=Ps_2sec, T=Q_(40, "°C"))
gas_rate = disch.Q()                                                                         #Fração mássica de gás após condensação
base_fluid_2sec = {k: v for k, v in zip(disch.fluid_names(), disch.mole_fractions_vapor())}         #composição do gás após condensação

new_suc_2sec = ccp.State(
    p=Ps_2sec, # original set 5.48 kgf/cm²
    T=Q_(40, "degC"),
    fluid=base_fluid_2sec
)

new_imp_2sec = ccp.Impeller.load_from_engauge_csv(
    suc=new_suc_2sec,
    curve_name="curve_2sec_custom",
    curve_path=data_dir,
    b=Q_(5, "mm"),
    D=Q_(294, "mm"),
    head_units="kJ/kg",
    flow_units="m³/h",
    power_losses_units = "kW",
    number_of_points=5,
)

flow_m_1 = res.x[0]
speed = imp_1sec.curves[1].speed
point_1sec = imp_1sec.point(flow_m= flow_m_1, speed=speed)

flow_m_2 = gas_rate*res.x[0]
speed = new_imp_2sec.curves[1].speed
new_point_2sec= new_imp_2sec.point(flow_m= flow_m_2, speed=speed)



In [11]:
book = xw.Book("WET_GAS_pressao_interestagio.xlsx")

def export_excel(new_point, column, sheet):
    
    mass_flow = new_point.flow_v * new_point.suc.rho()
    sheet.cells(4,column).value = mass_flow.to("kg/h").m

    sheet.cells(6,column).value = new_point.suc.p().to("kgf/cm²").m
    sheet.cells(7,column).value = new_point.suc.T().to("degC").m
    sheet.cells(8,column).value = new_point.suc.molar_mass().to("kg/kmol").m
    sheet.cells(9,column).value = new_point.flow_v.to("m**3/h").m

    sheet.cells(11,column).value = new_point.disch.p().to("kgf/cm²").m
    sheet.cells(12,column).value = new_point.disch.T().to("degC").m

    sheet.cells(14,column).value = new_point.power.to("kW").m
    #sheet.cells(15,column).value = new_point.power_shaft.to("kW").m
    sheet.cells(16,column).value = new_point.speed.to("rpm").m
    sheet.cells(17,column).value = new_point.head.to("kJ/kg").m
    sheet.cells(18,column).value = new_point.eff.m * 100
    #sheet.cells(19,column).value = new_point.torque.to("N*m").m

iteration = 0

export_excel(point_1sec, column = 3+(iteration*2),sheet = book.sheets["main"])
export_excel(new_point_2sec, column = 4+(iteration*2),sheet = book.sheets["main"])